# Exercise 1: Image classification

In [ ]:
from torch import nn
import torch
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Uncomment the device you want to use
# device = 'cpu'
device = 'cuda'
# device = 'mps'

## Model definition

In [ ]:
class ClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationModel, self).__init__()
        # CIFAR 3 in channel
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.relu4 = nn.ReLU()

        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # Flatten to 1d vector to prepare for classification task (e.g. 10 classes in CIFAR)
        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu4(x)

        x = self.fc2(x)

        return x




In [ ]:
# Test if model is working with dummy input
model = ClassificationModel(10)
print(model)
x = torch.randn(64,3,32,32)
print(model(x).shape)

ClassificationModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (relu4): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)
torch.Size([64, 10])


## Functions for model training and evaluation
### Don't edit the code in the cell below!

In [ ]:
def save_best_model(model, optimizer, epoch, loss, filename):
    print(f"Saving model at epoch {epoch} with accuracy {loss}")
    torch.save(
        {
            "model_state_dict": model.cpu().state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch,
            "loss": loss,
        },
        filename,
    )
    model.to(device)


def trainer(model, epochs, train_loader, val_loader, model_name="classification_model.pth"):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    best_accuracy = 0
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for i, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)
            loss = criterion(y_pred, y)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        print(f"Epoch: {epoch}, Loss: {total_loss/len(train_loader)}")

        model.eval()
        with torch.no_grad():
            total = 0
            correct = 0
            for i, (x, y) in enumerate(val_loader):
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                _, predicted = torch.max(y_pred, 1)
                total += y.size(0)
                correct += (predicted == y).sum().item()
            print(f"Epoch: {epoch}, Validation Accuracy: {correct / total}")
            if correct / total > best_accuracy:
                best_accuracy = correct / total
                save_best_model(
                    model, optimizer, epoch, best_accuracy, model_name
                )

def evaluate(model, val_loader):
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for i, (x, y) in enumerate(val_loader):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
        print(f"Accuracy: {correct / total}")

## Load the data

In [ ]:
# For exercise 1.3
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(90),
    # transforms.ColorJitter()
    ])


train_classification_dataset = CIFAR10("./data", download=True, transform=transform)
train_classification_loader = DataLoader(
    train_classification_dataset, batch_size=32, shuffle=True
)

validation_classification_dataset = CIFAR10(
    "./data", download=True, transform=transform, train=False
)
validation_classification_loader = DataLoader(
    validation_classification_dataset, batch_size=32, shuffle=False
)

Files already downloaded and verified
Files already downloaded and verified


## Train model

In [ ]:
model = ClassificationModel(10).to(device)
trainer(
    model,
    epochs=10, # If is needed increase the number of epochs
    train_loader=train_classification_loader,
    val_loader=validation_classification_loader,
)

Epoch: 0, Loss: 1.7825390793006541
Epoch: 0, Validation Accuracy: 0.4433
Saving model at epoch 0 with accuracy 0.4433
Epoch: 1, Loss: 1.4759778388402283
Epoch: 1, Validation Accuracy: 0.5028
Saving model at epoch 1 with accuracy 0.5028
Epoch: 2, Loss: 1.3502709624939673
Epoch: 2, Validation Accuracy: 0.5411
Saving model at epoch 2 with accuracy 0.5411
Epoch: 3, Loss: 1.2632217236764143
Epoch: 3, Validation Accuracy: 0.574
Saving model at epoch 3 with accuracy 0.574
Epoch: 4, Loss: 1.1923352714082178
Epoch: 4, Validation Accuracy: 0.5839
Saving model at epoch 4 with accuracy 0.5839
Epoch: 5, Loss: 1.1322832988075773
Epoch: 5, Validation Accuracy: 0.597
Saving model at epoch 5 with accuracy 0.597
Epoch: 6, Loss: 1.079333047834788
Epoch: 6, Validation Accuracy: 0.6112
Saving model at epoch 6 with accuracy 0.6112
Epoch: 7, Loss: 1.0368360036928075
Epoch: 7, Validation Accuracy: 0.6302
Saving model at epoch 7 with accuracy 0.6302
Epoch: 8, Loss: 0.9988607744223326
Epoch: 8, Validation Accur

## Evaluate model

In [ ]:
model = ClassificationModel(10).to(device)
checkpoint = torch.load("classification_model.pth")
model.load_state_dict(checkpoint["model_state_dict"])
evaluate(model, validation_classification_loader)

Accuracy: 0.6546
